### LearningRateDecay

In [ ]:
# import the necessary packages
import matplotlib.pyplot as plt
import numpy as np

class LearningRateDecay:
  pass

class StepDecay:
  def __init__(self, initAlpha=0.01, factor = 0.25, dropEvery=10):
    self.initAlpha = initAlpha
    self.factor = factor
    self.dropEvery = dropEvery


  def plot(self, epochs:list, title="Learning Rate Schedule"):
    lrs = [self(i) for i in epochs]

    plt.style.use('ggplot')
    plt.figure()
    plt.plot(epochs, lrs)
    plt.title(title)
    plt.xlabel('Epoch #')
    plt.ylabel('Learning Rate')

  def __call__(self, epoch):
    # compute the learning rate for the current epoch
    exp = np.floor((1 + epoch) / self.dropEvery)
    alpha = self.initAlpha * (self.factor ** exp)
    # return the learning rate
    return float(alpha)




class PolynomialDecay(LearningRateDecay):
  def __init__(self, maxEpochs=100, initAlpha=0.01, power=1.0):       # Note that if you set power=1.0 then you have a linear learning rate decay.
    # store the maximum number of epochs, base learning rate, and power of the polynomial
    self.maxEpochs = maxEpochs
    self.initAlpha = initAlpha
    self.power = power

  def __call__(self, epoch):
    # compute the new learning rate based on polynomial decay
    decay = (1 - (epoch / float(self.maxEpochs))) ** self.power
    alpha = self.initAlpha * decay
    # return the new learning rate
    return float(alpha)



In [ ]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from tensorflow.keras.applications.resnet50 import ResNet50
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np



epochs = 20
callbacks = []
schedule = StepDecay(initAlpha=1e-1, factor=0.25, dropEvery=15)
callbacks = [LearningRateScheduler(schedule)]

((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

labelNames = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

decay = 1e-1 / epochs

# initialize our optimizer and model, then compile it
opt =Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay= decay)

model = ResNet50(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(32,32,3),
    pooling="max",
    classes=10,
)



model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the network
H = model.fit(x=trainX, y=trainY, validation_data=(testX, testY),
	batch_size=128, epochs=epochs, callbacks=callbacks, verbose=1)

Epoch 1/20
391/391 [==============================] - 73s 72ms/step - loss: 3.4808 - accuracy: 0.1019 - val_loss: 2.3100 - val_accuracy: 0.1000 - lr: 0.1000
Epoch 2/20
391/391 [==============================] - 26s 67ms/step - loss: 2.2883 - accuracy: 0.1159 - val_loss: 3.5197 - val_accuracy: 0.0673 - lr: 0.1000
Epoch 3/20
391/391 [==============================] - 27s 70ms/step - loss: 1.9514 - accuracy: 0.2312 - val_loss: 2.0507 - val_accuracy: 0.2000 - lr: 0.1000
Epoch 4/20
391/391 [==============================] - 26s 67ms/step - loss: 1.8664 - accuracy: 0.2609 - val_loss: 2.3792 - val_accuracy: 0.1649 - lr: 0.1000
Epoch 5/20
391/391 [==============================] - 27s 68ms/step - loss: 1.8590 - accuracy: 0.2635 - val_loss: 2.6565 - val_accuracy: 0.1796 - lr: 0.1000
Epoch 6/20
391/391 [==============================] - 27s 68ms/step - loss: 2.2048 - accuracy: 0.2364 - val_loss: 1188.0146 - val_accuracy: 0.1009 - lr: 0.1000
Epoch 7/20
391/391 [==============================] - 2

In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=testX, batch_size=128)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=labelNames))
# plot the training loss and accuracy
N = np.arange(0, epochs)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on CIFAR-10")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig("train_plot")
# if the learning rate schedule is not empty, then save the learning
# rate plot
if schedule is not None:
	schedule.plot(N)
	plt.savefig("lr_plot")

[INFO] evaluating network...
79/79 [==============================] - 3s 14ms/step
              precision    recall  f1-score   support

    airplane       0.75      0.63      0.68      1000
  automobile       0.92      0.49      0.64      1000
        bird       0.89      0.12      0.21      1000
         cat       0.23      0.86      0.37      1000
        deer       0.67      0.49      0.57      1000
         dog       0.77      0.13      0.22      1000
        frog       0.92      0.43      0.59      1000
       horse       0.78      0.64      0.70      1000
        ship       0.59      0.91      0.71      1000
       truck       0.69      0.72      0.70      1000

    accuracy                           0.54     10000
   macro avg       0.72      0.54      0.54     10000
weighted avg       0.72      0.54      0.54     10000



<ipython-input-34-b36c3eb99d05>:19: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()
